In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, GRU, LSTM
import matplotlib.pyplot as plt

# Genearate Simple Timeseries Data

In [3]:
np.random.seed(42)
time = np.arange(0, 200, 0.1)
series = np.sin(0.1 * time) + 0.1 * np.random.randn(len(time))

In [6]:
time

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 1.997e+02, 1.998e+02,
       1.999e+02])

In [4]:
series

array([ 0.04967142, -0.0038266 ,  0.08476752, ...,  0.81210637,
        0.88829487,  0.83432859])

In [5]:
series.shape

(2000,)

# Create Supervised Data (Windowed Sequences)

In [7]:
window_size = 20 # How many past steps to use as input

X = []
y = []

for i in range(len(series) - window_size):
    X.append(series[i:i+window_size])
    y.append(series[i+window_size])

X = np.array(X)
y = np.array(y)


In [8]:
X

array([[ 0.04967142, -0.0038266 ,  0.08476752, ...,  0.20060708,
         0.08822717,  0.04762852],
       [-0.0038266 ,  0.08476752,  0.18229849, ...,  0.08822717,
         0.04762852,  0.34523421],
       [ 0.08476752,  0.18229849,  0.016574  , ...,  0.04762852,
         0.34523421,  0.18588227],
       ...,
       [ 0.89062322,  0.72459758,  0.84852516, ...,  0.92197703,
         0.99842373,  0.89324394],
       [ 0.72459758,  0.84852516,  0.71650806, ...,  0.99842373,
         0.89324394,  0.81210637],
       [ 0.84852516,  0.71650806,  0.68148436, ...,  0.89324394,
         0.81210637,  0.88829487]])

In [9]:
y

array([0.34523421, 0.18588227, 0.22498244, ..., 0.81210637, 0.88829487,
       0.83432859])

In [10]:
# RNN expects sahpe: [samples, timesteps, features]
X.shape

(1980, 20)

In [11]:
X = X[..., np.newaxis]

In [12]:
X.shape

(1980, 20, 1)

In [13]:
y.shape

(1980,)

# TRAIN TEST SPLIT

In [14]:
split_fraction = 0.8
split_index = int(len(X) * split_fraction)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (1584, 20, 1)
X_test shape: (396, 20, 1)


# BUILD the SimpleRNN Model

In [16]:
model = Sequential([
    SimpleRNN(32, input_shape=(window_size, 1)),
    Dense(1)
    ])

In [17]:
model.compile(optimizer='adam', loss='mse')

In [18]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.1302 - val_loss: 0.0133
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0132 - val_loss: 0.0126
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0132 - val_loss: 0.0132
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0134 - val_loss: 0.0123
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0138 - val_loss: 0.0124
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0135 - val_loss: 0.0121
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0133 - val_loss: 0.0121
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0126 - val_loss: 0.0129
Epoch 9/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0131 - val_loss: 0.0119
Epoch 10/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0134 - val_loss: 0.0121
Epoch 11/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0128 - val_loss: 0.0118
Epoch 12/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0127 - val_

In [22]:
series[-window_size:]

array([0.71650806, 0.68148436, 0.76246495, 0.916984  , 0.93158206,
       0.89301938, 0.91957753, 0.90395818, 0.79337937, 0.9058212 ,
       0.94768505, 0.94800163, 0.99630497, 0.95299741, 0.92197703,
       0.99842373, 0.89324394, 0.81210637, 0.88829487, 0.83432859])

# Forecast Next N Steps FROM the LAST Window(20)

In [21]:
n_forecast = 20

# Take the last window size points as the starting point
last_window = series[-window_size:]
current_input = last_window.reshape(1, window_size, 1)

forecast = []

for step in range(n_forecast):
  # Predict next values
  next_value = model.predict(current_input, verbose=0)[0, 0]
  forecast.append(next_value)

  # Slide the Window: drop first, append the new predicted value
  current_input = np.append(current_input[:, 1:, :], [[[next_value]]], axis=1)

print("\n Last Input window(Used for forecasting): ")
print(last_window)

print("\n Forecasted Next values: ")
print(forecast)


 Last Input window(Used for forecasting): 
[0.71650806 0.68148436 0.76246495 0.916984   0.93158206 0.89301938
 0.91957753 0.90395818 0.79337937 0.9058212  0.94768505 0.94800163
 0.99630497 0.95299741 0.92197703 0.99842373 0.89324394 0.81210637
 0.88829487 0.83432859]

 Forecasted Next values: 
[np.float32(0.862418), np.float32(0.86809033), np.float32(0.8553222), np.float32(0.8433761), np.float32(0.84288436), np.float32(0.8381213), np.float32(0.8208444), np.float32(0.81993484), np.float32(0.81290466), np.float32(0.8028676), np.float32(0.8050897), np.float32(0.79654497), np.float32(0.786538), np.float32(0.7853875), np.float32(0.7757052), np.float32(0.76662517), np.float32(0.7670712), np.float32(0.75870234), np.float32(0.75268567), np.float32(0.7478991)]
